# 2 Pruning 剪枝
## 2.1 Magnitude-Based Unstructured Pruning

### 2.1.1 数学原理

Magnitude pruning 假设：权重绝对值越小，对输出影响越弱，因此可以优先删除。这是一种典型的 **unstructured 剪枝** 方法。

设原始权重为 $w \in \mathbb{R}^d$，剪枝 mask 为 $m \in \{0,1\}^d$，剪枝后权重为 $w \odot m$。理想目标是：

$$
\min_m L(w \odot m) \quad \text{s.t.}\quad \|m\|_0 = k,
$$

其中 $\|m\|_0$ 是保留下来的权重个数（稀疏度约束）。这是一个组合优化问题，直接求解是 NP-hard。

Magnitude pruning 采用一个非常简单的启发式：

> 按权重的绝对值 $|w_i|$ 从小到大排序，删除最小的 $s\%$ 权重。

也可以写成：在所有大小为 $sd$ 的集合 $S$ 中，选择

$$
S^* = \arg\min_{S:\,|S| = sd} \sum_{i \in S} |w_i|,
$$

并将 $S^*$ 中的权重置零。

### 2.1.2 理论依据（Optimal Brain Damage 视角）

Optimal Brain Damage（LeCun, 1990）从二阶泰勒展开给出了权重重要性的经典形式。对损失 $L(w)$，对某个权重 $w_i$ 进行微小扰动：

$$
\Delta L_i \approx g_i \Delta w_i + \frac{1}{2} h_{ii} (\Delta w_i)^2,
$$

其中 $g_i = \partial L/\partial w_i$，$h_{ii}$ 是 Hessian 对角元素。若在收敛点附近，$g_i \approx 0$，并令剪枝时 $\Delta w_i = -w_i$，则有：

$$
\Delta L_i \approx \frac{1}{2} h_{ii} w_i^2.
$$

如果在同一层内 $h_{ii}$ 变化不大（或使用对角近似），则有

$$
\Delta L_i \propto w_i^2.
$$

因此，**权重绝对值越小，对损失的影响越小，越适合被剪掉**，这为 magnitude pruning 提供了一个简单但合理的理论基础。

### 2.1.3 Global vs Layerwise

- **Global pruning**：在全模型范围内对所有权重的 $|w_i|$ 做一次排序，删除总数中最小的 $s\%$。  
  - 更接近上面的优化形式，一般性能更好。

- **Layerwise pruning**：每一层单独按照 $|w_i|$ 排序，各自剪掉同样比例。  
  - 实现简单，但对有些层可能过度剪枝，对另一些层剪得太少，整体 suboptimal。

在工程实践中，global magnitude pruning 通常作为默认选择。

### 2.1.4 实际工程流程

1. 训练一个全精度基线模型，得到权重 $w^*$；  
2. 收集所有权重的绝对值 $|w_i^*|$；  
3. 在全模型范围按 $|w_i^*|$ 排序；  
4. 选择 sparsity 比例 $s$，删除最小的 $s\%$ 权重（对应位置的 $m_i = 0$）；  
5. 冻结剪枝结构，对剩余权重进行若干 epoch 的 fine-tune，恢复精度。

这就是最经典、最常用的 **unstructured magnitude pruning pipeline**。

### 2.1.5 文献

- **LeCun et al., *Optimal Brain Damage*, 1990**  
  <http://yann.lecun.com/exdb/publis/pdf/lecun-90b.pdf>  
  - 最早系统讨论基于权重重要性的连接剪枝，给出二阶泰勒展开与 Hessian 对角近似。

- **Molchanov et al., *Pruning Convolutional Neural Networks for Resource Efficient Inference*, ICLR 2017**  
  <https://arxiv.org/abs/1611.06440>  
  - 现代剪枝工作中常用的基线之一，明确将 “按权重绝对值排序后剪掉最小部分” 作为 magnitude-based pruning baseline。

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TinyMLP(nn.Module):
    def __init__(self, in_dim=784, hidden=256, out_dim=10):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden)
        self.fc2 = nn.Linear(hidden, out_dim)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


def global_magnitude_prune(model: nn.Module, sparsity: float):
    """
    对模型做一次 global magnitude 剪枝（只对 weight 参数）.

    Args:
        model: nn.Module
        sparsity: 要剪掉的比例 (0 ~ 1)，例如 0.5 表示剪掉 50% 最小权重
    """
    # 收集所有 weight
    weight_tensors = []
    for name, p in model.named_parameters():
        if "weight" in name:
            weight_tensors.append(p.data.view(-1))

    all_weights = torch.cat(weight_tensors)
    k = int(sparsity * all_weights.numel())
    print(f"{k=}, total weights={all_weights.numel()}")

    # kth-value 是从小到大排序后第 k 个元素
    threshold = all_weights.abs().kthvalue(k).values

    with torch.no_grad():
        for name, p in model.named_parameters():
            if "weight" in name:
                mask = (p.data.abs() > threshold).float()
                p.data.mul_(mask)


# quick smoke test
model = TinyMLP()
print("Total params before:", sum(p.numel() for p in model.parameters()))
global_magnitude_prune(model, sparsity=0.5)
print("Non-zero params after prune:",
      sum((p != 0).sum().item() for p in model.parameters() if p.dim() > 0))


Total params before: 203530
k=101632, total weights=203264
Non-zero params after prune: 101898


## 2.2 Taylor-Based Pruning（一阶 / 二阶近似）

### 2.2.1  数学原理

我们想估计：将某个参数 $w_i$ 置 0 时，损失会变化多少。

记 $L(w)$ 为平均损失，考虑只改动第 $i$ 维：

$$
\Delta L_i = L(w_1,\dots,0,\dots,w_d) - L(w)
$$

用一维 Taylor 展开近似：

$$
\Delta L_i \approx g_i \Delta w_i + \frac{1}{2} h_{ii} (\Delta w_i)^2
$$

其中

- $g_i = \frac{\partial L}{\partial w_i}$
- $h_{ii} = \frac{\partial^2 L}{\partial w_i^2}$
- 剪枝时令 $\Delta w_i = -w_i$

代入得到：

- 一阶近似：
  $$
  \Delta L_i^{(1)} \approx - g_i w_i
  $$
- 二阶近似：
  $$
  \Delta L_i^{(2)} \approx - g_i w_i + \frac{1}{2} h_{ii} w_i^2
  $$

若在「接近收敛」的点剪枝，一般有 $\mathbb{E}[g_i] \approx 0$，所以常用 **二阶项**：

$$
\Delta L_i^{(2)} \approx \frac{1}{2} h_{ii} w_i^2
$$

于是可以定义 **重要性分数(importance score，saliency score)**：

$$
I_i \propto h_{ii} w_i^2
$$

一阶版本（Molchanov）则使用

$$
I_i^{(1)} = \big|\mathbb{E}[g_i w_i]\big|
$$

在 mini-batch 上做平均。
该重要性指标可以按 weight、filter、channel 聚合，用于结构化剪枝。


在实践中：

- 精确 Hessian 很贵，常用 diagonal 近似或 Fisher 近似。
- 一阶版本（Molchanov）只需要梯度和权重，就能对 filter / channel 做结构化剪枝。

---

### 2.2.2. 工程落地：Hessian / Fisher 近似与实际流程

直接求 Hessian 非常昂贵，因此实践中常用近似。

#### Hessian diagonal（OBD / OBS）

- Optimal Brain Damage（LeCun, 1990）使用 Hessian 对角项 $h_{ii}$，提出了 $\Delta L_i \approx \tfrac{1}{2} h_{ii} w_i^2$ 的形式。  
- Optimal Brain Surgeon（Hassibi & Stork, 1993）进一步考虑 Hessian 逆矩阵，更精确但代价更大。

#### Fisher diagonal 近似（最常用替代 Hessian）

在很多工程系统中，用 **经验 Fisher 信息矩阵** 的对角来近似 Hessian：

$$
F_{ii} \approx \mathbb{E}\bigg[\Big(\frac{\partial L}{\partial w_i}\Big)^2\bigg].
$$

于是二阶分数可以写成

$$
I_i^{(F)} = F_{ii} w_i^2.
$$

优势：

- 不需要二阶梯度，只需在校准数据上统计梯度平方；  
- 与 EWC 等工作里“参数重要性”估计方法一致，工程上非常常见。


### 2.2.3. 实际可用算法（适合 CNN / Transformer 子模块）

一个可直接落地的 Taylor/Fisher 剪枝流程：

1. 训练好基线模型，固定参数 $w^*$；  
2. 准备一份代表真实分布的校准数据集（几百到几千个 batch 即可）；  
3. 在校准数据上多次前向 + 反向，累计  
   - 一阶分数：$\lvert g_i w_i \rvert$  
   - 或二阶（Fisher）分数：$g_i^2 w_i^2$；  
4. 将所有分数展开，做 **global 排序**；  
5. 按目标 sparsity（例如 50%）找到阈值，构造全局剪枝 mask；  
6. 应用剪枝（mask 掉对应权重或整条通道）；  
7. 用剪枝后的结构做若干 epoch 的 fine-tune，恢复性能。

如果要做 **结构化剪枝（channel/filter）**，可以把同一通道内所有权重的分数求和或求平均作为该通道的 importance，再对通道排序。

### 2.2.4 文献

- **LeCun et al., “Optimal Brain Damage”, NeurIPS 1989.**  
  <https://proceedings.neurips.cc/paper/1989/file/6c9882bbac1c7093bd25041881277658-Paper.pdf>

- **Hassibi & Stork, “Second Order Derivatives for Network Pruning: Optimal Brain Surgeon”, NeurIPS 1992.**  
  <https://proceedings.neurips.cc/paper/1992/file/303ed4c69846ab36c2904d3ba8573050-Paper.pdf>   

- **Amari, “Natural Gradient Works Efficiently in Learning”, Neural Computation, 1998.**  
  <https://www.semanticscholar.org/paper/Natural-Gradient-Works-Eciently-in-Learning-Amari/e1c2a2fd6a26947e5bbb8df47e30c1199ab1270d>

- **Kirkpatrick et al., “Overcoming Catastrophic Forgetting in Neural Networks”, PNAS 2017（EWC）.**  
  <https://www.pnas.org/doi/pdf/10.1073/pnas.1611835114>

- **Molchanov et al., “Pruning Convolutional Neural Networks for Resource Efficient Inference”, ICLR 2017.**  
  <https://arxiv.org/abs/1611.06440>

### 2.2.5 Toy example

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

class SmallCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

def collect_fisher_scores(model, dataloader, device="cpu"):
    """在一份校准数据上统计 Fisher-based Taylor 分数 g^2 * w^2."""
    model.to(device)
    model.train()

    fisher_scores = {
        name: torch.zeros_like(p)
        for name, p in model.named_parameters()
        if p.requires_grad and p.dim() > 1
    }

    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        model.zero_grad()
        out = model(x)
        loss = F.cross_entropy(out, y)
        loss.backward()

        with torch.no_grad():
            for name, p in model.named_parameters():
                if name in fisher_scores and p.grad is not None:
                    fisher_scores[name] += (p.grad ** 2) * (p.data ** 2)

    return fisher_scores

def collect_taylor_scores(model, dataloader, device="cuda"):
    model.eval()
    model.to(device)
    taylor_scores = {name: torch.zeros_like(p.data) for name, p in model.named_parameters() if "weight" in name}
    n_batches = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        model.zero_grad()
        out = model(x)
        loss = F.cross_entropy(out, y)
        loss.backward()
        with torch.no_grad():
            for name, p in model.named_parameters():
                if "weight" in name and p.grad is not None:
                    # 计算 |g * w| 一阶 Taylor 敏感度分数
                    taylor_scores[name].add_((p.grad * p).abs())
        n_batches += 1
    for name in taylor_scores:
        taylor_scores[name] /= max(1, n_batches)
    return taylor_scores


def prune_by_scores(model, score_dict, sparsity):
    """给定 importance score 字典，做一次 global 剪枝."""
    # 拉平成一个向量
    all_scores = torch.cat([s.view(-1) for s in score_dict.values()])
    k = int(all_scores.numel() * sparsity)
    # 选出要剪掉部分的最大分数作为阈值
    threshold = all_scores.kthvalue(k).values

    with torch.no_grad():
        for name, p in model.named_parameters():
            if name in score_dict:
                mask = (score_dict[name] > threshold).float()
                p.data.mul_(mask)


def evaluate(model, dataloader, device):
    model.eval()
    tot_loss, tot_correct, tot = 0.0, 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = F.cross_entropy(logits, y)
            tot_loss += loss.item() * y.size(0)
            preds = logits.argmax(dim=1)
            tot_correct += (preds == y).sum().item()
            tot += y.size(0)
    return {"loss": tot_loss / tot, "acc": tot_correct / tot}

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])
calib_ds_full = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
calib_ds = Subset(calib_ds_full, list(range(512)))  # use first 512 samples
calib_loader = DataLoader(calib_ds, batch_size=64, shuffle=False, num_workers=2)

val_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

model = SmallCNN(num_classes=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
orig_metrics = evaluate(model, val_loader, device)

scores = collect_taylor_scores(model, calib_loader, device=device)
prune_by_scores(model, scores, sparsity=0.5)
pruned_metrics = evaluate(model, val_loader, device)

# compare original model and pruned model
total_params = sum(p.numel() for p in model.parameters())
nonzero_params = sum((p != 0).sum().item() for p in model.parameters() if p.dim() > 0)
print(f"Total params: {total_params}, Non-zero after pruning: {nonzero_params}")

# ignore accuracy number comparision the dataset is too small. the difference is just random noise.
# But we can see with 50% weights pruned, the model size is halved. 
print("before:", orig_metrics)
print("after pruning by taylor score :", pruned_metrics)

Total params: 20042, Non-zero after pruning: 10074
before: {'loss': 2.305746308517456, 'acc': 0.1092}
after pruning by taylor score : {'loss': 2.3057271259307863, 'acc': 0.109}


In [17]:
model = SmallCNN(num_classes=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
orig_metrics = evaluate(model, val_loader, device)

scores = collect_fisher_scores(model, calib_loader, device=device)
prune_by_scores(model, scores, sparsity=0.5)
pruned_metrics = evaluate(model, val_loader, device)

# compare original model and pruned model
total_params = sum(p.numel() for p in model.parameters())
nonzero_params = sum((p != 0).sum().item() for p in model.parameters() if p.dim() > 0)
print(f"Total params: {total_params}, Non-zero after pruning: {nonzero_params}")

# ignore accuracy number comparision the dataset is too small. the difference is just random noise.
# But we can see with 50% weights pruned, the model size is halved. 
print("before:", orig_metrics)
print("after pruning by fisher score :", pruned_metrics)

Total params: 20042, Non-zero after pruning: 10074
before: {'loss': 2.3042822174072266, 'acc': 0.1103}
after pruning by fisher score : {'loss': 2.30426893119812, 'acc': 0.1102}


### 2.2.6 附：Fisher Information Matrix
#### - 深度学习中的损失（负对数似然）

分类模型优化的目标通常是最大化正确标签的概率：

$ p(y \mid x; \theta) $

负对数似然（即交叉熵）定义为：

$ L(\theta) = - \mathbb{E}_{(x,y)\sim \text{data}} \left[ \log p(y \mid x;\theta) \right] $

实际训练中用 mini-batch 近似：

$ L(\theta) \approx - \frac{1}{N} \sum_{i=1}^{N} \log p(y_i \mid x_i; \theta) $

---

#### - Fisher Information Matrix 的定义

Fisher 信息矩阵定义为 log-likelihood 梯度的二阶矩（外积的期望）：

$ F(\theta) = \mathbb{E}_{x,y} \left[ \nabla_\theta \log p(y \mid x; \theta) \, \nabla_\theta \log p(y \mid x; \theta)^\top \right] $

对角部分
$$F_{ii} = \mathbb{E} \left[\left(\frac{\partial}{\partial\theta_i}\log p(y\mid x;\theta)\right)^2\right]$$

它衡量：

**参数改变时，模型输出分布会变化多少。**

变化越大，参数越重要，Fisher 越大。换句话说，Fisher 是 log-likelihood 梯度的二阶矩 / 协方差矩

---

#### - 为什么 Fisher 可以近似 Hessian？

深度学习最常用的损失是负对数似然：

$ L(\theta) = - \log p(y \mid x;\theta) $
于是：
$$\frac{\partial L}{\partial\theta_i} = -\frac{\partial}{\partial \theta_i} \log p(y\mid x;\theta)$$

对比上面fisher 对角部分计算$\left(\frac{\partial}{\partial\theta_i}\log p(y\mid x;\theta)\right)^2$

$$F_{ii} = \mathbb{E} \left[\left(\frac{\partial}{\partial\theta_i}\log p(y\mid x;\theta)\right)^2\right]$$
就变成：
$$F_{ii} = \mathbb{E} \left[\left(\frac{\partial L}{\partial\theta_i}\right)^2\right]$$

- Fisher diagonal 可以直接用 loss 对权重的梯度平方估计
- 所以剪枝代码里会看到 (grad ** 2) 而不是 grad_log_prob。

在最大似然估计理论中有一个关键结论：

$ H(\theta) = \nabla_\theta^2 L(\theta) \approx F(\theta) $

原因如下：

- 在最优点，梯度期望为 0  
- 对 log-likelihood 的二阶导在数学上与梯度外积的期望等价  
- 因此 Hessian 和 Fisher 在收敛区域具有相同结构

这称为 **Hessian–Fisher 等价性**。

因此：

$ h_{ii} \approx F_{ii} $

---

#### - 为什么剪枝中使用 Fisher diagonal？

Taylor 二阶展开给出的权重重要性：

$ \Delta L_i^{(2)} \approx \frac{1}{2} h_{ii} w_i^2 $

若 Hessian diagonal 难计算，可用 Fisher diagonal 替代：

$ F_{ii} \approx \mathbb{E} \left[ \left( \frac{\partial L}{\partial w_i} \right)^2 \right] $

于是重要性分数变为：

$ I_i = F_{ii} \, w_i^2 $

这是现代剪枝（如 Molchanov 2017）与连续学习（EWC 2017）的统一基础。

---

#### - 工程上如何计算 Fisher diagonal

对校准数据执行多次 forward/backward，并累积：

$ F_{ii} \approx \frac{1}{N} \sum_{n=1}^{N} g_{i,n}^2 $

其中 $ g_{i,n} $ 为第 n 个 batch 的梯度。

特点：

- 无需二阶梯度  
- 无需 Hessian-vector product  
- 与训练流程兼容  
- 可应用于 CNN / Transformer / MLP

---

#### - 工业应用总结

- **剪枝（Taylor-based / Movement pruning）**：参数或通道重要性  
- **量化（如 SmoothQuant）**：激活/权重量化敏感度  
- **连续学习（EWC）**：参数重要性  
- **自然梯度优化**：Fisher 作为参数空间中的 Riemann 度量  

Fisher diagonal 是深度学习中最常用的“近似二阶信息”。

## 2.3 — Global vs Layerwise & LAMP（Layer-Adaptive Magnitude Pruning）

### 2.3.1. 问题：每一层剪多少？

Magnitude/Taylor 给了每个权重/通道的分数，但没有回答：

- 每一层应该剪多少？
- 均匀剪？深层剪得多一点？还是手调一堆超参？

**LAMP**（Layer-Adaptive Magnitude-based Pruning）的目标：

> 用一个简单的重标定（rescaling），让 global magnitude 排序自然带出合理的「层自适应 sparsity」。

---

### 2.3.2. LAMP 核心思想

- 把某一层的权重 $W$ 展平成一维，并按绝对值从小到大排序：
$$|W[1]| \le |W[2]| \le \dots \le |W[n]|$$
- 对排序之后的第 $u$ 个元素，定义
$$\text{score}(u; W) = \frac{W[u]^2}{\sum_{v \ge u} W[v]^2}$$

也就是**“这个权重的能量 / 剩下所有还没被剪掉的能量”**

直觉：

- 本来就「能量很集中、冗余少」的层，如果再剪，会 hurt 得更多，因此其分数被放大。
- 冗余很多的层分数被压缩，可以多剪一些。

然后做的事很简单：

- 在每一层算出这个 score；

- 把所有层的 score 扔在一起，做一次 global 排序 + 全局阈值；

- 阈值以下的都剪掉，就自动给出了每层该剪多少（layer-adaptive）。

结果：直接对 $s_i$ 做 global 排序剪枝，就会自动给出「层自适应」的配额。

### 2.3.3 算法
#### 2.3.3.1 单层情况
对单层全连接 $y = W x$，如果我们要在固定 sparsity $\kappa$ 下找一个 mask $M$（0/1 矩阵）去剪枝：

剪完之后是 $W_f = M \odot W$。

想要最小化最坏情况下的输出 $ℓ_2$ 误差：
$$\min_{M, \|M\|_0 \le \kappa} \ \sup_{\|x\|_2 \le 1} \|W x - (M\odot W) x\|_2$$
用谱范数的定义 + 上界，可以把它 relax 到 Frobenius norm
$$\approx \min_{M,\|M\|_0 \le \kappa} \|W - M\odot W\|_F$$

这一步的结论：
在单层上，要在 Frobenius 失真最小的前提下选 $\kappa$ 个保留的 weight，显然就是「保留 $|W_{ij}|$ 最大的那几个」——也就是 layerwise magnitude pruning 本身。https://arxiv.org/pdf/2010.07611

所以 MP（magnitude pruning）可以被理解成：

对每一层，求解“在给定层内 sparsity 下，使权重的 $ℓ_2$ 失真最小”的最优方案

#### 2.3.3.2 整个网络：模型级输出失真上界
真正想要的是：在给定全局 sparsity 下，最小化模型输出的 $ℓ_2$ 失真：
$$\min_{\{M^{(i)}\}} \sup_{\|x\|_2\le 1} \big\| f(x; W^{(1:d)}) - f(x; W_f^{(1:d)})\big\|_2$$
其中 $W^{(i)}$ 是第 $i$ 层权重，$W_f^{(i)} = M^{(i)}\odot W^{(i)}$ 是剪掉后的权重.

直接解这个太难，于是他们搞了一个贪心 + 上界:

- 假设每次只剪掉一个权重；

- 推出一个模型输出误差的上界（这里用了很多不等式和层间范数的乘积）：
$$\begin{array}{c} \sup_{\|x\|_2\le 1} 
\big\| f(x; W^{(1:d)}) - f(x; W^{(1:i-1)}, W_f^{(i)}, W^{(i+1:d)})\big\|_2
\\
\le
\frac{\|W^{(i)} - W_f^{(i)}\|_F}{\|W^{(i)}\|_F} \cdot \prod_{j=1}^{d}\|W^{(j)}\|_F \end{array}$$
右边除了 $|W^{(i)} - W_f^{(i)}|_F / |W^{(i)}|_F$ 之外，其他都是常数，它们对「剪谁」这件事不产生排序影响。

于是问题变成：

每剪掉一个 weight，希望让「本层相对失真」 $|W^{(i)} - W_f^{(i)}|_F / |W^{(i)}|_F$ 的增量尽量小

#### 2.3.3.3 LAMP score 的形式
由此推出 LAMP score 的形式

把一层的权重按从小到大排序后，想象一个从小到大逐个剪掉的过程：

- 当前在位置 $u$ 上的 weight 是 $W[u]$；

- 假设之前 $1,\dots,u-1$ 都已经被剪掉了；

- 剩余的能量是 $\sum_{v\ge u} W[v]^2$；

- 这时候如果再剪掉 $W[u]$，其带来的本层失真增量和 $W[u]^2$ 成正比；

- 但「相对坏程度」取决于这个增量占当前剩余量的比例。

所以就得到：
$$\text{score}(u; W) = \frac{W[u]^2}{\sum_{v\ge u} W[v]^2}$$
解释：

- 分子：当前这条连接的能量；

- 分母：剪掉它之前本层剩余还活着的总能量（把之前更小的都视为已经剪掉了）；

也就是说：

- 如果你再剪这一条，会把剩余的能量吃掉多少比例？

- 高分的权重 = 一旦剪掉，本层「最后一点血」会掉很多 → 更该被保留；
- 低分的权重 = 所处层还很富裕 / 冗余多，可以多剪。

然后在全网络上，我们只需要：

- 对每层都算这个 score；

- 用一个全局 threshold 统一剪枝；

- 结果就是：自然出现「某些层被多剪，某些层被少剪」，而不需要手调各层 sparsity。

#### pytorch LAMP score

In [18]:
import torch


def lamp_LAMP_scores(w: torch.Tensor):
    """
    根据原始论文的数学定义计算 LAMP 分数。
    Score(W_u) = W_u^2 / Sum(W_v^2 for all v >= u)
    返回同形状的 score 张量。
    """
    # 1. 展平张量
    flat = w.view(-1)

    # 2. 按绝对值从小到大排序
    abs_w, idx = flat.abs().sort()

    # 3. 计算排序后权重的平方
    sq = abs_w.pow(2)

    # 4. 计算平方后的前缀和 (从小到大累加)
    cumsum = torch.cumsum(sq, dim=0)

    # 5. 计算剩余能量（总能量 - 前缀和）
    total_energy = sq.sum()
    # remaining_energy_sum[i] 代表排在第 i 位（含）之后所有权重的能量和
    remaining_energy_sum = total_energy - cumsum + sq # 加上 sq[i] 因为 cumsum[i] 减去了它

    # 6. 计算分数：当前权重的平方 除以 剩余能量总和
    # 为了数值稳定性，添加一个小的 epsilon
    epsilon = 1e-8
    # 分子：当前权重平方 (sq)，分母：剩余能量和 (remaining_energy_sum)
    scores_sorted = sq / (remaining_energy_sum + epsilon)

    # 7. 还原到原始张量的形状和位置
    inv_idx = torch.empty_like(idx)
    inv_idx[idx] = torch.arange(len(idx), device=w.device)
    flat_scores = scores_sorted[inv_idx]
    
    return flat_scores.view_as(w)


def compute_lamp_scores(model: nn.Module):
    lamp_scores = {}
    for name, p in model.named_parameters():
        if "weight" in name:
            lamp_scores[name] = lamp_scores_for_tensor(p.data)
    return lamp_scores


# 使用方式：
# 1. scores = compute_lamp_scores(model)
# 2. prune_by_scores(model, scores, sparsity=0.8)



### 2.3.4 文献
Lee et al., *Layer-adaptive Sparsity for the Magnitude-based Pruning*, NeurIPS 2020.  
<https://arxiv.org/abs/2010.07611>

### 2.3.5 LAMP 的学术地位与工业落地情况
在 **2021 年之后的剪枝文献中**，LAMP 经常被当作  
- 「magnitude-based global pruning 下的 layer-adaptive sparsity baseline」，  
- 用来和信息论、概率图、门控机制、拓扑方法等更复杂的剪枝策略做对比

对比当前（2022–2025）主流压缩 / 部署方向

引用最近的模型压缩综述与剪枝论文：

- **A Survey on Model Compression for Deep Neural Networks (2023)**  
  🔗 https://arxiv.org/abs/2308.07610  

- **What’s Left? Pruning LLMs (2023)**  
  🔗 https://arxiv.org/abs/2302.10403  

- **SparseGPT（LLM 剪枝主流方法）**  
  🔗 https://arxiv.org/abs/2301.00774  

- **Wanda: Weights Are Not Equal（LLM 剪枝另一个主流）**  
  🔗 https://arxiv.org/abs/2306.11695  

这些工作有如下共识：

---

 ✔ Quantization 是工业落地核心（INT8 / FP8 / BF16）

相比重度剪枝，量化更稳定、收益更大：

- INT8：显著减带宽  
- FP8：NVIDIA Hopper 推动成为新推理标准  
- BF16：车端/云端最常见的 mixed precision format  

---

✔ 真要剪枝，各模型的主流方法如下：

 **① CNN / Vision：Structured Pruning + Distillation**

- Channel / Filter / Block 剪枝  
- 辅以蒸馏来保持精度  

示例：  
- **Network Slimming**  
  🔗 https://arxiv.org/abs/1708.06519  

- **DMCP**  
  🔗 https://arxiv.org/abs/2005.03354  

---

**② Transformer / LLM：Movement Pruning / SparseGPT / Wanda**

Transformer 对 unstructured magnitude pruning（包括全局 magnitude + LAMP 打分）非常脆弱，因此业界转向更稳定的剪枝方式：

- **Movement Pruning（Google, 2020）**  
  🔗 https://arxiv.org/abs/2005.07683  

- **SparseGPT（2023）**  
  🔗 https://arxiv.org/abs/2301.00774  

- **Wanda（2023）**  
  🔗 https://arxiv.org/abs/2306.11695  

这些方法在 70–90% sparsity 下的稳定性远优于 magnitude-based 剪枝。

---

**③ N:M 稀疏（2:4）是真正能加速的稀疏模式**

- 支持 NVIDIA Ampere/Hopper 的 sparse tensor core  
- 是唯一真正能带来吞吐提升的稀疏方式  
- 广泛用于工业推理引擎  
- 也可以结合 LAMP 做 **per-layer group-level sparsity allocation**

相关论文：  
**Learning N:M Fine-grained Structured Sparse Neural Networks From Scratch**  
🔗 https://arxiv.org/abs/2111.10988  

---
综合结论（学术 vs 工业）


 🏭 工业落地角度  
LAMP **不是工业界的最终剪枝方案**，而是：

- 一种 **layer-wise sparsity allocator**  
- 常用于分析哪些层冗余大、哪些层不能碰  
- 可能与 N:M、structured pruning、QAT、distillation 等结合使用  

真正上线的 pipeline 一般是：

1. **Structured pruning（channel/head/block）**  
2. **N:M 稀疏（2:4）**  
3. **量化（INT8, FP8, BF16）**  
4. **蒸馏恢复精度**  
5. 针对硬件的 **自定义 kernel / compiler pass**

而不是：

> 「仅靠 LAMP 做 90% unstructured pruning 后直接部署」。

---

**一句话总结：**

> **LAMP = 强大的层自适应 sparsity 分配器（academic SOTA baseline），但不是工业端最终的高性能剪枝方案。**  
> 工业界更依赖 structured/N:M sparsity + quantization + hardware-specific kernels。

## 2.4 Structured Pruning（通道 / Filter 剪枝）
### 2.4.1. 非结构化 vs 结构化

- **非结构化剪枝**：单个权重变 0，参数矩阵变稀疏，但形状不变。
- **结构化剪枝**：直接删除整个通道 / filter / head / block，模型形状发生改变：
  - 卷积层：删除输出通道（或输入通道）。
  - 全连接层：删除一整列 / 一整行。
  - Transformer：删除整个 attention head / FFN hidden block。

结构化剪枝的优点：

- FLOPs 真正下降，容易被 TensorRT / ONNX Runtime / TVM 等编译器利用。
- 硬件友好（不需要特殊稀疏 kernel 也能加速）。

---

### 2.4.2 通道 / Filter 剪枝的数学视角

以一个卷积层为例，忽略 bias：

- 输入：$X\in\mathbb{R}^{N\times C_{\text{in}}\times H\times W}$
- 卷积核：$W\in\mathbb{R}^{C_{\text{out}}\times C_{\text{in}}\times k_h\times k_w}$
- 输出：$Y\in\mathbb{R}^{N\times C_{\text{out}}\times H'\times W'}$

删除输出通道 $c$：

- 等价于将 $W_{c,:,:,:}$ 置零，并在实现上真正删除该通道。
- 其影响可以用某种「通道重要性分数」来衡量，例如：
  $$
  I_c = \|W_c\|_2 \quad\text{或}\quad I_c = |\gamma_c|
  $$
  其中 $\gamma_c$ 是该通道对应的 BatchNorm 缩放系数（Network Slimming）。

### 2.4.3 python 例子

In [19]:
import torch.nn.utils.prune as prune

def structured_channel_prune(conv_layer: nn.Conv2d, amount: float = 0.3):
    """
    对单个 Conv2d 层进行结构化剪枝，在 dim=0 (输出通道) 维度上删掉 L2 范数最小的 amount 比例通道.
    """
    prune.ln_structured(conv_layer, name='weight', amount=amount, n=2, dim=0)
    # 将 mask 应用为永久（删除 pruning 相关 buffer）
    prune.remove(conv_layer, 'weight')


def structured_prune_model(model: nn.Module, amount: float = 0.3):
    """对模型中所有 Conv2d 层进行统一结构化剪枝."""
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            structured_channel_prune(m, amount=amount)


# 使用方式：
# cnn_model = ...
# structured_prune_model(cnn_model, amount=0.5)

> 更高级用法：
>
> - 用 BN 的 $\gamma_c$ 或 Taylor 分数来对每个通道打分，决定删哪些通道，而不是对每层用固定 `amount`。
> - 对 Transformer，可以对 attention head 或 FFN hidden 维度做类似的「block-level」剪枝。

---

### 2.4.4 文献

- Liu et al., **Learning Efficient Convolutional Networks through Network Slimming**, ICCV 2017.  
  <https://arxiv.org/abs/1708.06519>
- Molchanov et al. 系列（Taylor-based structured pruning）：  
  <https://arxiv.org/abs/1611.06440>  
  <https://arxiv.org/abs/1906.10771>

PyTorch 官方剪枝文档：  
<https://pytorch.org/docs/stable/nn.utils.prune.html>

---

### 2.4.5 工业落地备注

- 对 CNN backbone 做结构化剪枝可以直接带来 GFLOPs 的下降，在 TensorRT / ONNX / TVM 上常常能看到 1.5–3× 的 latency 改善（视 sparsity 和 kernel 实现而定）。
- 对 Transformer，可以：
  - 剪掉部分 attention heads；
  - 减小 FFN hidden size；
  - 删除某些层（layer dropping / layer dropping with distillation）。
- 对车端部署来说，结构化剪枝是比非结构化更直接、可控的降 FLOPs 手段。

## 2.5 N:M / 半结构化稀疏（以 2:4 Sparsity 为例）

### 2.5.1 约束形式

以 NVIDIA Ampere 的 **2:4 sparsity** 为例：

- 在每 4 个连续权重（通常沿某个矩阵维度）中，**最多 2 个非零**。
- 等价描述：在每个长度为 4 的 block 中，只允许 2 个位置是非零。


记权重矩阵 $W\in\mathbb{R}^{M\times N}$，在每一行的 block $b\in\mathbb{R}^4$ 上约束：

$$
\|b\|_0 \le 2
$$

整个剪枝问题可以写成：

$$
\min_{W} L(W) \quad \text{s.t.}\; \forall b,\; \|b\|_0 \le 2
$$

#### 2.5.1.1 block  是从 𝑊 里“切片”的一个 4 元组
设$W\in\mathbb{R}^{M\times N}$选定一个方向（最常见：沿列方向，也就是固定行、取连续 4 个列元素）：
- 取第 𝑖 行，从第 4𝑘 列开始的连续 4 个元素：
$$b_{i,k} = [W_{i,4k},\; W_{i,4k+1},\; W_{i,4k+2},\; W_{i,4k+3}] \in \mathbb{R}^4$$
- 你也可以沿行方向切，或者在卷积里沿特定展平后的维度切。关键是：“连续的 4 个”要跟底层库/硬件的格式约定一致，否则就算你满足 2:4，也不会触发硬件加速。[nvidia developer](https://developer.nvidia.com/blog/exploiting-ampere-structured-sparsity-with-cusparselt)

#### $\|b\|_0$ 范数
$\ell_0$“范数”其实不是严格意义的范数，它定义为：
$$\|b\|_0 = \#\{j \in \{1,2,3,4\} \mid b_j \neq 0\}$$
也就是：b 里有几个元素不为 0。

#### 为什么这叫 “2:4 sparsity”：它是一种“半结构化（semi-structured）”稀疏
- “非结构化稀疏”：哪里都可以为 0，mask 很乱，硬件很难高效跳过。
- 2:4 半结构化稀疏”：你只能在每个 4 元组里把 2 个变成 0（或保留 2 个非零），pattern 被限制，硬件能用固定格式把它压缩并用专门 kernel 加速. 很多实现还会更进一步，要求：
$\|b\|_0 = 2$ 严格 50% 稀疏，这样压缩格式更固定、元数据更好管理。Ampere Sparse Tensor Cores 的路线基本是围绕这种“固定 2-of-4”格式做的
---

### 2.5.2 简单的 2:4 剪枝算法（离线）

典型工程做法：

1. 对每个 block 的 4 个权重计算绝对值。
2. 在 block 内保留 top-2 较大的，其他设为 0。
3. 在全模型或某些层上统一执行，再 fine-tune 若干 epoch。

---

### 2.5.3. PyTorch 示例：对 Linear 层做 2:4 blockwise 剪枝


In [20]:
def prune_2_4_blockwise(weight: torch.Tensor, dim: int = 1):
    """
    对给定权重张量做 2:4 剪枝（简化版）.

    假设 dim=1 (列方向)，N 必须是 4 的倍数。
    在每 (M, 4) block 中保留绝对值最大的 2 个元素.
    """
    w = weight.data
    if dim == 0:
        w = w.t()  # 交换成 (in, out) 方便处理

    M, N = w.shape
    assert N % 4 == 0, "For this simple example, N must be multiple of 4."

    w_view = w.view(M, N // 4, 4)

    with torch.no_grad():
        abs_block = w_view.abs()
        # 在 block 内选 top-2
        top2_idx = abs_block.topk(k=2, dim=-1).indices
        mask = torch.zeros_like(w_view, dtype=torch.bool)
        mask.scatter_(-1, top2_idx, True)
        w_view *= mask

    # 写回
    if dim == 0:
        weight.data.copy_(w.t())
    else:
        weight.data.copy_(w)


# 使用：
# linear = nn.Linear(1024, 1024, bias=False)  # 这里 out_features 必须 %4==0
# prune_2_4_blockwise(linear.weight, dim=1)


> 注意：
>
> - 真实系统中，2:4 模式会与专门的 Sparse Tensor Core kernel 配合使用。
> - block 的划分方式（按行/列、沿哪个维度）要和硬件 / 底层库的约定一致。

---

### 4. 推荐阅读

- NVIDIA, **Accelerating Inference with Sparsity Using the NVIDIA Ampere Architecture**  
  <https://developer.nvidia.com/blog/accelerating-inference-with-sparsity-using-the-nvidia-ampere-architecture/>
- PyTorch Blog, **Accelerating Inference with Sparsity in PyTorch**（2:4 支持）  
  <https://pytorch.org/blog/accelerating-inference-with-sparsity/>

  - 直接把权重某些位置设为 0，张量还是 dense，默认 GEMM kernel 仍然会照算，所以不会变快。

  - 要变快，你得把权重转换到 semi-structured sparse 的压缩格式，并走专门 kernel（PyTorch 内部/后端会做相应 dispatch）
---

### 5. 工业落地备注

- 在 A100 / H100 上，2:4 稀疏矩阵乘可以获得最高 2× 的理论加速，实际端到端加速视网络结构和 I/O 开销，大约在 1.3–1.8×。
- 如果未来车端 SoC 提供类似 N:M 稀疏矩阵指令，你在这里积累的「block-constrained 剪枝 + fine-tune」经验可以直接迁移过去。

## 2.6 Movement Pruning & 动态稀疏（Fine-tuning 场景）

### 2.6.1 背景：从「大小」到「移动趋势」

在下游任务 fine-tune 场景（例如从大预训练模型到特定任务）：

- 某些权重最终数值小，但在 fine-tune 过程中一直在往「有用」方向移动。
- 某些权重最终数值大，但其实在 fine-tune 过程中被往 0 拉。

**Movement Pruning** 的核心：

> 不仅看权重的大小，还看在 fine-tune 过程中的「移动方向 / 移动强度」，对 mask 的分数做「随训练更新」。

参考：Sanh et al., *Movement Pruning: Adaptive Sparsity by Fine-Tuning for NLP*, NeurIPS 2020.  
<https://arxiv.org/abs/2005.07683>

---

### 2.6.2. 概念框架（简化）

为每个权重引入一个「评分变量」 $s_i$，mask 定义为：

$$
m_i = \mathbf{1}(s_i > 0)
$$

优化目标：

$$
\min_{w,s} L\big(w\odot m(s)\big) + \lambda\|s\|_1
$$

- 用 STE（Straight-Through Estimator）来训练 $m(s)$。
- 在训练过程中，$s_i$ 会根据梯度「移动」：
  - 对任务重要的权重，其对应的 $s_i$ 会被推向正值（保留）；
  - 不重要的权重，其 $s_i$ 会被推向负值（剪掉）。

Movement Pruning 的分数本质上是：

> 在 fine-tune 的整个过程中，权重被推向「保留」还是「剪掉」方向的 **累积趋势**。
Movement Pruning = 👉 把“剪 or 不剪”这件事，也变成一个可学习的连续优化变量


1️⃣ 传统 magnitude pruning 的假设
$|w_i|$ 小 ⇒ 不重要 ⇒ 可以剪

隐含假设：

权重的“当前数值大小” ≈ “它对任务的重要性”

⚠️ 在 fine-tuning 场景，这个假设经常是错的：

预训练权重很大
→ fine-tune 时被持续往 0 拉（说明：不适合新任务）

某些权重当前很小
→ 但梯度一直在把它往同一个方向推（说明：正在变重要）

👉 重要的不是“现在多大”，而是“训练在把它往哪里推”

#### 2.6.2.1 $s_i$ 意义
评分变量 $s_i$ 不是权重本身，而是：
👉「这个权重 是否值得存在 的可学习证据累积器」
- $w_i$: 这个权重“取什么值”
- $s_i$: 这个权重“要不要被保留”

Movement Pruning =
👉 把“剪 or 不剪”这件事，也变成一个可学习的连续优化变量

#### 2.6.2.2 $s_i$ 为什么$|w_i|$在 fine-tune 里不够
1️⃣ 传统 magnitude pruning 的假设:
$|w_i|$ 小 ⇒ 不重要 ⇒ 可以剪
隐含假设：

权重的“当前数值大小” ≈ “它对任务的重要性”
⚠️ 在 fine-tuning 场景，这个假设经常是错的：

预训练权重很大
→ fine-tune 时被持续往 0 拉（说明：不适合新任务）

某些权重当前很小
→ 但梯度一直在把它往同一个方向推（说明：正在变重要）

👉 重要的不是“现在多大”，而是“训练在把它往哪里推”

#### 2.6.2.3 权重在训练中三个方面的衡量
| 变量    | 含义       | 角色             |
| ----- | -------- | -------------- |
| $w_i$ | 权重值      | 数值大小 / 表达能力    |
| $s_i$ | **评分变量** | 是否值得保留         |
| $m_i$ | mask     | 真正参与计算的 0/1 开关 |

关系是：
$$m_i = \mathbf{1}(s_i > 0)$$
$$\text{effective weight} = w_i \cdot m_i$$

- 为什么要有 $s_i$, 不能直接学$m_i$?
  - 因为：$m_i \in \{0,1\}$, **离散、不可导**
  - 训练是靠梯度的 → 没法直接优化

#### 2.6.2.4 优化目标
$$\min_{w,s} \; L\big(w \odot m(s)\big) + \lambda \|s\|_1$$


- 任务Loss $L(w \odot m(s))$
forward 时：
  - 如果 $s_i > 0$ 这个权重参与计算
  - 如果 $s_i \le 0$ 这个权重被屏蔽
📌 注意：loss 并不直接看到 $s_i 但会通过 mask 间接影响 $s_i$

- 稀疏化：$\lambda \|s\|_1$
  - $\|s\|_1$ 越小
  - 越多$s_i \to 0$
  - 越多 mask 被关掉
👉 和 LASSO 非常像，但对象不是 $w$ 而是“是否保留”  
	​
#### 2.6.2.5 梯度如何“推动 $s_i$?
- mask 不可导，于是用：Straight-Through Estimator (STE)。直觉就是：

  - forward：👉 hard threshold（0 / 1）

  - backward：假装 $m_i \approx s_i$
  - 让梯度直接流到 $s_i$

- 梯度如何影响 $s_i$
假设某次反向传播中：

梯度希望 增大 $w_i$ （有助于降低 loss），当前 $w_i > 0$ ；那么梯度会推动$s_i$↑（让 mask 更稳地为 1）； 
反之，如果梯度方向与 $w_i$ 当前符号相反，或者该权重对 loss 没帮助 👉 $s_i$会被推向负值 → mask 关闭


---

### 2.6.3. PyTorch 示例：简化的 movement-like 分数

下面是一个「玩具版」的 movement-style 分数收集器：用累计的 $-\text{sign}(w)\cdot g$ 作为 movement 近似。


In [21]:
def collect_movement_scores(model, dataloader, device="cuda", steps=100):
    """
    简化版 movement score:
    对每个 weight 统计 sum (-grad * sign(weight)) / steps.
    分数越大，说明在训练过程中越被往当前符号方向推动，越重要。
    """
    model.to(device)
    model.train()

    movement_scores = {
        name: torch.zeros_like(p.data)
        for name, p in model.named_parameters() if "weight" in name
    }

    opt = torch.optim.SGD(model.parameters(), lr=1e-3)

    it = iter(dataloader)
    for step in range(steps):
        try:
            x, y = next(it)
        except StopIteration:
            it = iter(dataloader)
            x, y = next(it)

        x, y = x.to(device), y.to(device)
        opt.zero_grad()
        out = model(x)
        loss = F.cross_entropy(out, y)
        loss.backward()

        with torch.no_grad():
            for name, p in model.named_parameters():
                if "weight" in name and p.grad is not None:
                    movement_scores[name].add_(-p.grad * p.data.sign())

        opt.step()

    for name in movement_scores:
        movement_scores[name] /= max(1, steps)

    return movement_scores


# 使用：
# scores = collect_movement_scores(model, train_loader, steps=200)
# prune_by_scores(model, scores, sparsity=0.5)


`movement_scores[name].add_(-p.grad * p.data.sign())`, 数学的解读是：
因为权重更新的梯度下降
- SGD 更新：$w \leftarrow w - \eta g$
- Movement score 用的是：$\text{score} \;\sim\; - g \cdot \mathrm{sign}(w)$


$$\Delta s_i \;\propto\; - g_i \cdot \text{sign}(w_i)$$

$$s_i \approx \sum_t - g_i^{(t)} \cdot \mathrm{sign}(w_i^{(t)})$$
这等价于：

统计整个 fine-tune 过程中：
梯度有多少次在“支持它继续存在”

- 偶尔支持 → 分数小

- 长期支持 → 分数大

- 长期反对 → 分数负 → 剪掉

| 情况      | $g_i$        | $\text{sign}(w_i)$ | $-g_i \cdot \text{sign}(w_i)$ | 解读       |
| ------- | ------------ | ------------------ | ----------------------------- | -------- |
| 梯度想增大权重 | 与 sign(w) 相反 | +                  | 正                             | **支持保留** |
| 梯度想减小权重 | 与 sign(w) 同号 | +                  | 负                             | **支持剪掉** |


👉 累计的是：
“训练过程中，梯度有多大程度在支持这个权重继续存在”

这就是为什么：Movement score ≈ “保留证据的时间积分”

> 说明：
>
> - 这不是原论文的精确实现，只是有类似「movement 意图」的简化版示例，方便在小模型上玩。
> - 真正的 Movement Pruning 使用显式的 mask 参数化、梯度裁剪等，更适合用现成实现（如 HuggingFace 的实现）。

Movement Pruning 的 $s_i$ 是：

“训练动力学视角的 importance”

| 方法           | 回答的问题          |
| ------------ | -------------- |
| Magnitude    | 现在值大不大         |
| Hessian      | 被扰动会不会炸        |
| Jacobian     | 输入噪声会不会放大      |
| **Movement** | **训练是否持续想保留它** |

---



### 2.6.4. 推荐阅读

- Sanh et al., **Movement Pruning: Adaptive Sparsity by Fine-Tuning for NLP**, NeurIPS 2020.  
  <https://arxiv.org/abs/2005.07683>
- HuggingFace 实现（支持 BERT 等）：  
  <https://github.com/huggingface/transformers/tree/main/examples/research_projects/movement-pruning>

---

### 2.6.5. 工业落地备注

- 对大型 Transformer（尤其 NLP / 多模态）在车端做 domain-specific fine-tune 时，可以使用 Movement Pruning：
  - 不需要从头训练 sparse 模型。
  - 直接在下游任务 fine-tune 阶段收敛到一个带稀疏的权重配置。
- 对比纯 magnitude：Movement Pruning 更「任务 aware」，更适合下游微调场景。